In [1]:
import os, os.path 
import numpy 
from torch.utils.data import Dataset, DataLoader
import pickle
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import TensorDataset, DataLoader

"""Change to the data folder"""
train_path = "C:/Users/Kennnnnnn/Desktop/CSE151B/CSE151 FINAL/new_train/new_train"
test_path = 'C:/Users/Kennnnnnn/Desktop/CSE151B/CSE151 FINAL/new_val_in/new_val_in'
matplotlib.rcParams['savefig.dpi'] = 300

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform
        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
    def __len__(self): #len(val_dataset)
        return len(self.pkl_list)
    def __getitem__(self, idx): #val_dataset[0]
        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        if self.transform:
            data = self.transform(data)
        return data

In [3]:
train_dataset = ArgoverseDataset(data_path=train_path) 
test_dataset  = ArgoverseDataset(data_path=test_path) 

# Data Preprocessing

In [56]:
len(train_dataset) *0.7

144159.4

## Data Splitting

we want to perform 70/30 split on training dataset since the test dataset has no label

In [57]:
#diff points
#This is the training set after splitting
train = []
for i in range(144159):
    val = train_dataset[i]
    c_filter = val['car_mask'].reshape(-1) == 1
    p_in = val['p_in'][c_filter]
    p_out = val['p_out'][c_filter]
    for c in range(len(p_in)):
        in_out = np.diff(np.vstack((p_in[c], p_out[c])), axis=0).flatten()
        train.append(in_out)
train = np.array(train)

In [80]:
#diff points
#This is the testing set after splitting
test_30 = []
for i in range(144159,len(train_dataset)):
    val = train_dataset[i]
    c_filter = val['car_mask'].reshape(-1) == 1
    p_in = val['p_in'][c_filter]
    p_out = val['p_out'][c_filter]
    for c in range(len(p_in)):
        in_out = np.diff(np.vstack((p_in[c], p_out[c])), axis=0).flatten()
        test_30.append(in_out)
test_30 = np.array(test_30)

# Training

## Auto-regression model set up

In [24]:
#Step up device: cuda or cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [59]:
class Net(nn.Module):
    
    def __init__(self, column_size):
        
        super().__init__()
        
        self.size_lin1 = column_size // 2
        self.size_lin2 = self.size_lin1 // 3

        self.lin1 = nn.Linear(column_size, self.size_lin1)
        self.lin2 = nn.Linear(self.size_lin1, self.size_lin2)
        self.lin3 = nn.Linear(self.size_lin2, 1)
        
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        return self.activation(self.lin3(x))
    

## Training with auto regression

In [58]:
net_list = []

for i in range(36, 36+60):
    
    net_list.append(Net(i).to(device))
    
for num in tqdm(range(60)):

    in_train = torch.tensor(train[:, :36 + num]).type(torch.FloatTensor).to(device)
    out_train = torch.tensor(train[:, 36: 36+ 1 + num]).type(torch.FloatTensor).to(device)

    trainloader = DataLoader(TensorDataset(in_train[:, :36 + num], out_train[:, num: num+1]),
                             batch_size= 512, drop_last=False)
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(net_list[num].parameters(), lr=0.01)

    for epoch in range(5): 
    
        net_list[num].train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data 
            # zero the parameter gradients
            optimizer.zero_grad()
            outputs = net_list[num](inputs).to(device)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


In [196]:
num_parameters = 0
for i in range(36, 36+60):
    num_parameters += sum(p.numel() for p in Net(i).parameters())
num_parameters

161945

## Prediction

first of all, in_test should always be a numpy array

In [61]:
#re run it for numpy array
init_p = []
in_test = []
for i in range(144159,len(train_dataset)):
    val = train_dataset[i]
    c_filter = val['agent_id'] == val['track_id'][:, 0, 0]
    p_in = val['p_in'][c_filter]
    v_in = (val['v_in'][c_filter][:, :, 0]**2 + val['v_in'][c_filter][:, :, 1] ** 2)**0.5
    for c in range(len(p_in)):
        in_test.append(
            np.diff(p_in[c], axis=0).flatten()
        )
        init_p.append(
            p_in[c][-1]
        )
in_test = np.array(in_test)
init_p = np.array(init_p)

Creating prediction

In [62]:
for i in tqdm(range(60)):
    in_test_tensor = torch.tensor(in_test).type(torch.FloatTensor).to(device)
    pred = net_list[i](in_test_tensor).to('cpu').detach().numpy().flatten()
    in_test = np.insert(in_test, 36 + i, pred, axis=1)

In [63]:
pred = in_test[:, 36:]

In [67]:
# This is the coordinates of prediction
x_pred = pred[:, list(range(0, 60, 2))]
y_pred = pred[:, list(range(1, 61, 2))]

In [83]:
#This is the corrdinates of actual number
x_actu = test_30[:, list(range(0, 60, 2))]
y_actu = test_30[:, list(range(1, 61, 2))]

In [120]:
x_pred[:, 0] = x_pred[:, 0] + init_p[:, 0]

In [121]:
y_pred[:, 0] = y_pred[:, 0] + init_p[:, 1]

In [122]:
x_pred = x_pred.cumsum(1)
y_pred = y_pred.cumsum(1)

In [197]:
df = np.zeros((61783, 60))
num = 0
for i in range(30):
    df[:, num] = x_pred[:, i]
    df[:, num + 1] = y_pred[:, i]
    num += 2

## Prediction result (for the last 30% training data)

In [125]:
df = pd.DataFrame(df)
df

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,519.105957,4000.403564,519.105957,4000.403584,519.105957,4000.403586,519.105957,4000.403891,519.105957,4000.404453,...,519.105958,4000.507278,519.105974,4000.507283,519.105974,4000.509291,519.105974,4000.509330,519.105974,4000.510473
1,718.107215,3023.205566,718.139924,3023.205566,718.146638,3023.205566,718.157298,3023.205566,718.157395,3023.205566,...,718.204318,3023.350831,718.208757,3023.350831,718.208791,3023.350831,718.208851,3023.441877,718.209566,3023.441930
2,3688.480469,2097.048340,3688.480469,2097.048340,3688.480469,2097.048340,3688.480469,2097.048340,3688.480469,2097.048340,...,3688.480469,2097.049900,3688.480469,2097.049900,3688.480469,2097.049901,3688.480469,2097.049901,3688.480469,2097.109196
3,195.968857,2339.386085,195.968857,2339.386153,195.968857,2339.386316,195.968857,2339.386595,195.968857,2339.393230,...,195.969002,2339.444640,195.969004,2339.444701,195.969005,2339.444801,195.969006,2339.444831,195.969006,2339.444871
4,3186.894775,1889.668213,3186.894775,1889.668213,3186.894775,1889.668213,3186.894775,1889.668213,3186.894775,1889.668213,...,3186.894775,1889.672423,3186.894793,1889.672423,3186.894794,1889.672424,3186.894794,1889.672424,3186.894794,1889.703598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61778,523.651774,3858.902733,523.762627,3859.573696,523.920278,3860.241814,524.055569,3860.932777,524.203237,3861.580684,...,525.688789,3874.999037,525.750705,3875.614632,525.766796,3876.254452,525.793379,3876.824876,525.825593,3877.446784
61779,3392.507324,1921.914054,3392.507324,1922.058464,3392.507324,1922.158442,3392.507324,1922.338184,3392.507324,1922.452399,...,3392.507379,1924.422764,3392.507838,1924.429501,3392.507838,1924.434881,3392.507900,1924.437811,3392.507900,1924.445368
61780,739.490719,2264.093994,739.493630,2264.093994,739.516462,2264.093994,739.534547,2264.093994,739.535376,2264.093994,...,739.626426,2264.095698,739.627331,2264.095698,739.627900,2264.095698,739.627905,2264.095699,739.627926,2264.096025
61781,2826.473145,1306.424316,2826.473145,1306.424316,2826.473145,1306.424316,2826.473145,1306.424316,2826.473145,1306.424316,...,2826.473145,1306.431980,2826.473204,1306.431980,2826.473205,1306.432003,2826.473205,1306.432004,2826.473205,1306.440201


In [27]:
#df.insert(0, 'ID', pd.read_csv('sample_submission.csv')['ID'])

In [28]:
#df.columns = pd.read_csv('sample_submission.csv').columns

In [29]:
#df.to_csv('submission.csv', index=False)